# Sessie 04 -  Natural Language Processing

Sinds enkele jaren schieten de Natural Language Processing toepassingen als paddenstoelen uit de grond. Eén van de belangrijkste redenen daarvoor is de steeds groter wordende beschikbaarheid van tekstuele data aangeleverd door bijvoorbeeld social media.

In deze opdracht zal NLP toegepast worden in het kader van sentiment analysis. De bedoeling van sentiment analysis is om uit een tekstueel bericht zoals een email, sms, twitter bericht, Trip Advisor review het sentiment te voorspellen. Bedrijven weten bijvoorbeeld graag wat het sentiment is in de berichten die online over hen verschijnen.

De opdracht bestaat uit 2 deelopdrachten.

De eerste deelopdracht gaat rond het uitvoeren van pure sentiment analysis van Tweets.
Bij de tweede deelopdracht is het de bedoeling om cyber trolls op te sporen.

Telkens wordt het bag-of-words model gebruikt om tekst voor te stellen en vervolgens worden verschillende classifiers getraind om het sentiment/rating te voorspellen.

Het bag-of-words model beschrijft het voorkomen van woorden binnen een document en doet dit op bais van een vocabulair van gekende woorden en meet de aanwezigheid van deze gekende woorden. De grote beperking van het bag-of-words model is dat het enkel rekening houdt met welke woorden in een bericht voorkomen en niet met de volgorde ervan. Daardoor is het bag-of-words model heel beperkt in het capteren van de context waarin woorden in een bericht voorkomen . Tijdens de module deep learning zullen we afstappen van het bag-of-words model en gebruik maken van word embeddings in combinatie met LSTM neurale netwerken. Op deze manier kunnen we modellen trainen die wel rekening houden met de context en woordvolgordes.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

import nltk
import seaborn as sns
from nltk.corpus import stopwords
import re #regular expressions
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_colwidth',150)

## Airline sentiment

De dataset 'Airlines_sentiment.csv' bevat tweets over verschillende airlines. Jouw taak bestaat erin om een classifier te trainen die zo goed mogelijk het sentiment van de tweets kan classificeren als positive, neutral en negative.
Het sentiment bevindt zich in de kolom 'airline_sentiment' en de tweet zelf in de kolom 'text'.
Doorloop de volgende stappen:
- Analyse van de data
- Preprocessing
- Training en hyperparameter tuning
- Modeloptimalisatie


In [2]:
# Inlezen van de dataset

dataset = pd.read_csv('Airline_sentiment.csv',encoding = 'ISO-8859-1')
dataset.tail()

_unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
14635  681679794    False   finalized                   3     2/25/15 19:46   
14636  681679795    False   finalized                   3     2/25/15 19:14   
14637  681679796    False   finalized                   3     2/25/15 19:04   
14638  681679797    False   finalized                   3     2/25/15 18:59   
14639  681679798    False   finalized                   3     2/25/15 19:06   

      airline_sentiment  airline_sentiment:confidence          negativereason  \
14635          positive                        0.3487                     NaN   
14636          negative                        1.0000  Customer Service Issue   
14637           neutral                        1.0000                     NaN   
14638          negative                        1.0000  Customer Service Issue   
14639           neutral                        0.6771                     NaN   

       negativereason:confidence   airline airline_sentiment_gold  \
14635                     0.0000  American                    NaN   
14636                     1.0000  American                    NaN   
14637                        NaN  American                    NaN   
14638                     0.6659  American                    NaN   
14639                     0.0000  American                    NaN   

                  name negativereason_gold  retweet_count  \
14635  KristenReenders                 NaN              0   
14636         itsropes                 NaN              0   
14637         sanyabun                 NaN              0   
14638       SraJackson                 NaN              0   
14639        daviddtwu                 NaN              0   

                                                                                                                                                        text  \
14635                                                                                        @AmericanAir thank you we got on a different flight to Chicago.   
14636  @AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer...   
14637                                                                                           @AmericanAir Please bring American Airlines to #BlackBerry10   
14638                @AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??   
14639             @AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?   

      tweet_coord  tweet_created      tweet_id tweet_location  \
14635         NaN  2/22/15 12:01  5.695880e+17            NaN   
14636         NaN  2/22/15 11:59  5.695870e+17          Texas   
14637         NaN  2/22/15 11:59  5.695870e+17  Nigeria,lagos   
14638         NaN  2/22/15 11:59  5.695870e+17     New Jersey   
14639         NaN  2/22/15 11:58  5.695870e+17     dallas, TX   

                    user_timezone  
14635                         NaN  
14636                         NaN  
14637                         NaN  
14638  Eastern Time (US & Canada)  
14639                         NaN

### Analyse van de dataset

- Onderzoek de gebalanceerdheid van de dataset. Stel grafisch de verdeling voor van het sentiment.
- Visualiseer de verdeling van het sentiment per airline. Maak daarvoor per airline een Seaborn countplot van het sentiment.
- Welke airline lijkt op basis van deze tweet het best te scoren en welke het slechtst?
- Uit hoeveel woorden bestaat het langste twitter bericht en uit hoeveel woorden het kortste?

In [ ]:
# Analyse van de dataset


### Preprocessing

Onderzoek of er mogelijks foutieve of ontbrekende data aanwezig is. 
Kuis de tweets op: verwijder stopwoorden en niet-letters, zet alles om naar lowercase en pas stemming toe.
Maak gebruik van de CountVectorizer en TfidfTransformer om een bag-of-words model te creëren. 

Meer info: 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer 

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

Splits op in een training set en test set. Zorg ervoor dat er 3000 tweets in de test set steken.


In [ ]:
# Trainen van de classifiers (naive bayes, logistic regression, SVM)


Welke classifier heeft jouw voorkeur? Beargumenteer in termen van accuracy, f1-score maar ook de berekeningstijd.


In [ ]:
# Antwoord


### Modeloptimalisatie

- Stel dat de airlines vooral geïnteresseerd zijn in het correct opsporen van negatieve tweets. Welke aanpassingen zou je kunnen doen om ervoor te zorgen dat het model minder negatieve tweets verkeerd classificeert? Test deze aanpassingen.

- Bekom je een beter model met een hogere accuraatheid wanneer je ook de feature airline in rekening brengt? Een optie hierbij is ook om voor elke airline een apart model te trainen. 

- Bekom je een beter model wanneer je de lengte van de tweets in rekening brengt?

In [ ]:
# Modeloptimalisatie


##  Cyber trolls

Een cyber troll heeft als bedoeling emotionele reacties uit te lokken door opzettelijk verkeerde, kwetsende en agressieve berichten te plaatsen op forums, newsgroups, social media, etc. Online platformen hebben er een groot belang bij om deze cyber trolls te kunnen detecteren en weren van hun platform. Daarvoor gebruiken ze geavanceerde nlp algoritmes. Ontwerp zelf een cyber troll detector op basis. Je kan daarvoor beroep doen op de dataset 'Cybetrolls.csv'. Om de dataset te evalueren gebruik je een test set van 5000 berichten.

Welke woorden worden het meest door cyber trolls gebruikt? Geef een top 10.
Welke woorden dragen voor een logistic classifier het meest bij tot klasse 'troll bericht'? Geef ook hier een top 10.

Voor het evalueren van de classifier gebruik je de volgende metrics: accuracy, f1 score en de ROC. Formuleer conclusies. Bijvoorbeeld naar de keuze toe van de classifier, interpretatie van de ROC of auROC. Toon enkele verkeerd geclassificeerde berichten en probeer te achterhalen waarom ze verkeerd geclassificeerd werden.





In [4]:
dataset = pd.read_csv('Cybetrolls.csv')
dataset.head()

annotation  \
0           1   
1           1   
2           1   
3           1   
4           1   

                                                                                                                              content  
0                                                                                                              Get fucking real dude.  
1   She is as dirty as they come  and that crook Rengel  the Dems are so fucking corrupt it's a joke. Make Republicans look like  ...  
2   why did you fuck it up. I could do it all day too. Let's do it when you have an hour. Ping me later to sched writing a book here.  
3       Dude they dont finish enclosing the fucking showers. I hate half assed jobs. Whats the reasononing behind it? Makes no sense.  
4                                                          WTF are you talking about Men? No men thats not a menage  that's just gay.